## Deliverable 2. Create a Customer Travel Destinations Map.

In [37]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gmaps
import requests
from citipy import citipy
# Import the API key.
from config import weather_api_key
from config import g_key
from datetime import datetime

In [38]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,71.64,86,44,5.01,light rain
1,1,Saint-Philippe,RE,-21.3585,55.7679,69.39,75,60,12.10,broken clouds
2,2,Hilo,US,19.7297,-155.0900,85.28,80,90,13.80,light rain
3,3,Port Alfred,ZA,-33.5906,26.8910,48.79,77,88,32.28,moderate rain
4,4,Leningradskiy,RU,69.3833,178.4167,38.66,75,100,5.23,overcast clouds


In [39]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 80


In [40]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,71.64,86,44,5.01,light rain
1,1,Saint-Philippe,RE,-21.3585,55.7679,69.39,75,60,12.10,broken clouds
3,3,Port Alfred,ZA,-33.5906,26.8910,48.79,77,88,32.28,moderate rain
7,7,Bluff,NZ,-46.6000,168.3333,46.80,63,77,10.40,broken clouds
8,8,Orzhytsya,UA,49.7918,32.7012,61.61,91,96,8.48,overcast clouds
10,10,Rikitea,PF,-23.1203,-134.9692,72.70,63,18,13.29,few clouds
11,11,Vaini,TO,-21.2000,-175.2000,78.96,78,20,9.22,few clouds
12,12,Kuching,MY,1.5500,110.3333,77.22,88,40,3.44,moderate rain
14,14,Busselton,AU,-33.6500,115.3333,48.04,74,67,6.87,broken clouds
17,17,Yellowknife,CA,62.4560,-114.3525,73.33,60,75,11.50,smoke


In [41]:
# 4a. Determine if there are any empty rows.
len(preferred_cities_df)

495

In [42]:
np.where(pd.isnull(preferred_cities_df))

(array([ 55,  56,  79, 163, 299, 406, 410]), array([2, 2, 2, 2, 2, 2, 2]))

In [43]:
preferred_cities_df.replace('', np.nan)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,71.64,86,44,5.01,light rain
1,1,Saint-Philippe,RE,-21.3585,55.7679,69.39,75,60,12.10,broken clouds
3,3,Port Alfred,ZA,-33.5906,26.8910,48.79,77,88,32.28,moderate rain
7,7,Bluff,NZ,-46.6000,168.3333,46.80,63,77,10.40,broken clouds
8,8,Orzhytsya,UA,49.7918,32.7012,61.61,91,96,8.48,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
685,685,Wuwei,CN,37.9281,102.6414,79.47,40,58,1.52,broken clouds
686,686,Mugur-Aksy,RU,50.3500,90.5000,59.07,63,45,2.82,scattered clouds
687,687,Plettenberg Bay,ZA,-34.0527,23.3716,47.79,74,100,24.05,light rain
690,690,Tupik,RU,54.4253,119.9327,60.53,94,100,0.60,overcast clouds


In [44]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_prefer_city_df = preferred_cities_df.dropna()
new_prefer_city_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,71.64,86,44,5.01,light rain
1,1,Saint-Philippe,RE,-21.3585,55.7679,69.39,75,60,12.10,broken clouds
3,3,Port Alfred,ZA,-33.5906,26.8910,48.79,77,88,32.28,moderate rain
7,7,Bluff,NZ,-46.6000,168.3333,46.80,63,77,10.40,broken clouds
8,8,Orzhytsya,UA,49.7918,32.7012,61.61,91,96,8.48,overcast clouds
10,10,Rikitea,PF,-23.1203,-134.9692,72.70,63,18,13.29,few clouds
11,11,Vaini,TO,-21.2000,-175.2000,78.96,78,20,9.22,few clouds
12,12,Kuching,MY,1.5500,110.3333,77.22,88,40,3.44,moderate rain
14,14,Busselton,AU,-33.6500,115.3333,48.04,74,67,6.87,broken clouds
17,17,Yellowknife,CA,62.4560,-114.3525,73.33,60,75,11.50,smoke


In [45]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_prefer_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,71.64,light rain,42.6001,-73.9662,
1,Saint-Philippe,RE,69.39,broken clouds,-21.3585,55.7679,
3,Port Alfred,ZA,48.79,moderate rain,-33.5906,26.8910,
7,Bluff,NZ,46.80,broken clouds,-46.6000,168.3333,
8,Orzhytsya,UA,61.61,overcast clouds,49.7918,32.7012,
10,Rikitea,PF,72.70,few clouds,-23.1203,-134.9692,
11,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,
12,Kuching,MY,77.22,moderate rain,1.5500,110.3333,
14,Busselton,AU,48.04,broken clouds,-33.6500,115.3333,
17,Yellowknife,CA,73.33,smoke,62.4560,-114.3525,


In [57]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # USED _ hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    #IndexError
    #Recall that when trying to parse the data from an API request, we need to use a try-except block to handle the error with a statement and continue the retrieval.
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [58]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,71.64,light rain,42.6001,-73.9662,
1,Saint-Philippe,RE,69.39,broken clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
3,Port Alfred,ZA,48.79,moderate rain,-33.5906,26.8910,The Halyards Hotel
7,Bluff,NZ,46.80,broken clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
8,Orzhytsya,UA,61.61,overcast clouds,49.7918,32.7012,


In [59]:
new_name_df = hotel_df[hotel_df["Hotel Name"]!='']

In [60]:
len(hotel_df)

488

In [61]:
len(new_name_df)

434

In [62]:
dropped_hotel_df = hotel_df.replace('', np.nan)

In [63]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = new_name_df
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saint-Philippe,RE,69.39,broken clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
3,Port Alfred,ZA,48.79,moderate rain,-33.5906,26.8910,The Halyards Hotel
7,Bluff,NZ,46.80,broken clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
10,Rikitea,PF,72.70,few clouds,-23.1203,-134.9692,People ThankYou
11,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,Keleti Beach Resort


In [64]:
# 8a. Create the output File (CSV)
#output_data_file = "../Weather_Database/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
#clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [65]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

In [66]:
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

In [67]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))